# **KLASIFIKASI EMOSI menggunakan metode K-Nearest Neighbor**

dataset diambil dari : https://www.kaggle.com/datasets/ejlok1/toronto-emotional-speech-set-tess?resource=download
Dataset audio ini akan di ektraksi menjadi beberapa fitur numeric ZCR dan RMSE.
Dataset ini berisi serangkaian 200 kata target yang diucapkan dalam frasa pembawa "Katakan kata _" oleh dua aktris (berusia 26 dan 64 tahun), dan rekaman dibuat untuk setiap kata dengan menggambarkan tujuh emosi berbeda (marah, jijik, takut, bahagia, kaget menyenangkan, sedih, dan netral). Totalnya terdapat 2800 titik data (file audio) dalam dataset ini.

Penyusunan dataset ini dilakukan dengan cara bahwa setiap dari dua aktris perempuan dan emosi yang mereka tunjukkan terkandung dalam folder tersendiri. Di dalamnya, terdapat file audio untuk semua 200 kata target. Format file audio ini adalah format WAV.

Dengan kata lain, dataset ini memberikan akses ke rekaman suara dari dua aktris perempuan yang berbeda usia, masing-masing mengucapkan 200 kata target dalam tujuh berbagai emosi. Setiap kombinasi kata dan emosi memiliki rekaman suara sendiri-sendiri, dan semuanya tersusun dengan baik dalam struktur folder yang jelas. Format file audio yang digunakan adalah WAV.



# **Instalasi Library Librosa**

In [ ]:
pip install librosa

**Menghubungkan ke google drive**

In [ ]:
# Import library drive dari modul google.colab untuk mengakses Google Drive
from google.colab import drive
# Mount Google Drive ke dalam direktori /content/drive di Google Colab
drive.mount('/content/drive')


Mounted at /content/drive


1. from google.colab import drive: Baris ini mengimpor modul drive dari paket google.colab. Modul ini menyediakan fungsionalitas untuk mengakses Google Drive dari Google Colab.

2. drive.mount('/content/drive'): Baris ini memanggil fungsi mount dari modul drive. Fungsi ini digunakan untuk mengaitkan Google Drive dengan sesi Google Colab. Argumen '/content/drive' adalah direktori tempat Google Drive akan dimount di dalam sesi Colab.

In [ ]:
%cd /content/drive/MyDrive/PSD/emosi

/content/drive/MyDrive/PSD/emosi


1. %cd: Ini adalah magic command di Google Colab yang digunakan untuk mengganti direktori kerja.

2. /content/drive/MyDrive/PSD/emosi: Ini adalah path (alamat) dari direktori yang akan dijadikan direktori kerja. Dengan menjalankan baris ini, direktori kerja sekarang akan diubah menjadi /content/drive/MyDrive/PSD/emosi.

Jadi, setelah menjalankan baris ini, semua operasi file dan folder selanjutnya akan dilakukan di dalam direktori /content/drive/MyDrive/PSD/emosi pada Google Drive.

In [ ]:
# Import library untuk berinteraksi dengan sistem operasi (OS)
import os
# Import pustaka librosa untuk analisis sinyal suara
import librosa
# Import pustaka numpy untuk manipulasi data numerik
import numpy as np
# Import pustaka pandas untuk analisis dan manipulasi data tabular
import pandas as pd
# Import beberapa fungsi statistik dari pustaka scipy
from scipy.stats import skew, kurtosis, mode

1. import os: Pustaka ini memungkinkan program Python berinteraksi dengan sistem operasi (OS), seperti membuat, menghapus, atau memindahkan file dan direktori.

2. import librosa: Librosa adalah pustaka Python yang umum digunakan untuk analisis sinyal suara. Ini menyediakan fungsi untuk ekstraksi fitur suara, pemrosesan suara, dan tugas-tugas terkait suara lainnya.

3. import numpy as np: NumPy adalah pustaka dasar untuk komputasi numerik di Python. Di sini, kita mengimpor NumPy dan memberinya alias np, yang umum digunakan sebagai konvensi.

4. import pandas as pd: Pandas adalah pustaka Python yang digunakan untuk analisis dan manipulasi data tabular. DataFrame, struktur data utama Pandas, sering digunakan untuk menyimpan dan mengelola data.

5. from scipy.stats import skew, kurtosis, mode: Scipy adalah pustaka ilmiah untuk Python, dan di sini, kita mengimpor beberapa fungsi statistik seperti skewness, kurtosis, dan mode dari pustaka scipy.stats. Fungsi ini digunakan untuk menghitung beberapa statistik deskriptif dari distribusi data.

In [ ]:
#variael yang menyimpan list nama folder yang berisi dataset emosi
folders=['YAF_sad','YAF_pleasant_surprised','YAF_neutral',
         'YAF_happy','YAF_fear','YAF_disgust','YAF_angry',
         'OAF_sad','OAF_pleasant_surprise','OAF_neutral',
         'OAF_happy','OAF_fear','OAF_disgust',
         'OAF_angry',
         ]

**Ektraksi Audio menjadi fitur numerik**

In [ ]:
def calculate_statistics(audio_path):
    # Menggunakan librosa untuk memuat file audio
    y, sr = librosa.load(audio_path)

    # MENGHITUNG NILAI ZERO-CROSSING RATE (ZCR)
    zcr_mean = np.mean(librosa.feature.zero_crossing_rate(y=y))
    zcr_median = np.median(librosa.feature.zero_crossing_rate(y=y))
    zcr_std_dev = np.std(librosa.feature.zero_crossing_rate(y=y))
    zcr_kurtosis = kurtosis(librosa.feature.zero_crossing_rate(y=y)[0])
    zcr_skew = skew(librosa.feature.zero_crossing_rate(y=y)[0])

    # MENGHITUNG NILAI ROOT MEAN SQUARE ERROR (RMSE)
    rmse = np.sum(y**2) / len(y)
    rmse_median = np.median(y**2)
    rmse_std_dev = np.std(y**2)
    rmse_kurtosis = kurtosis(y**2)
    rmse_skew = skew(y**2)

    # return daftar nilai statistik
    return [zcr_mean, zcr_median, zcr_std_dev, zcr_kurtosis, zcr_skew, rmse, rmse_median, rmse_std_dev, rmse_kurtosis, rmse_skew]

1. y, sr = librosa.load(audio_path): Menggunakan pustaka librosa untuk memuat file audio dari path yang diberikan. y adalah time series data suara, dan sr adalah tingkat sampel (sampling rate) dari audio.

2. Perhitungan statistik Zero-Crossing Rate (ZCR):
zcr_mean: Nilai rata-rata Zero-Crossing Rate.
zcr_median: Nilai median Zero-Crossing Rate.
zcr_std_dev: Standar deviasi Zero-Crossing Rate.
zcr_kurtosis: Nilai kurtosis dari distribusi Zero-Crossing Rate.
zcr_skew: Nilai skewness dari distribusi Zero-Crossing Rate.
Perhitungan statistik Root Mean Square Error (RMSE):
3. perhitungan RMSE
rmse: Nilai rata-rata Root Mean Square Error.
rmse_median: Nilai median Root Mean Square Error.
rmse_std_dev: Standar deviasi Root Mean Square Error.
rmse_kurtosis: Nilai kurtosis dari distribusi Root Mean Square Error.
rmse_skew: Nilai skewness dari distribusi Root Mean Square Error.
4. return daftar nilai-nilai statistik. Nilai-nilai ini dapat digunakan untuk menganalisis karakteristik sinyal suara dan dapat dimasukkan ke dalam struktur data yang sesuai, seperti DataFrame dari Pandas.

In [ ]:
features =[] #variabel yang akan digunakan untuk fitur

In [ ]:
for folder in folders:
    folder_path = f'{folder}'
    for filename in os.listdir(folder_path):
        if filename.endswith('.wav'):
            audio_path = os.path.join(folder_path, filename)
            statistics = calculate_statistics(audio_path)
            features.append([folder] + statistics)

1. for folder in folders:: Loop ini akan iterasi melalui setiap elemen dalam variabel folders. Setiap elemen mungkin merupakan nama folder yang berisi file audio.

2. folder_path = f'{folder}': Ini mendefinisikan variabel folder_path yang merupakan path lengkap ke folder saat ini. Pada contoh ini, sepertinya tidak diperlukan karena folder seharusnya sudah berisi path yang lengkap.

3. for filename in os.listdir(folder_path):: Ini mengiterasi melalui setiap file dalam folder yang ditentukan oleh folder_path.

4. if filename.endswith('.wav'):: Pernyataan ini memastikan bahwa hanya file dengan ekstensi .wav yang akan diproses.

5. audio_path = os.path.join(folder_path, filename): Ini menggabungkan folder_path dan filename untuk mendapatkan path lengkap ke file audio.

7. statistics = calculate_statistics(audio_path): Memanggil fungsi calculate_statistics untuk menghitung statistik dari file audio yang sedang diproses.

8. features.append([folder] + statistics): Menambahkan hasil statistik ke dalam list features. Di sini, [folder] + statistics digunakan untuk menyertakan informasi folder (kategori) sebagai bagian dari data fitur yang dikumpulkan.

In [ ]:
columns =  ['Label']+['ZCR Mean', 'ZCR Median', 'ZCR Std Dev', 'ZCR Kurtosis', 'ZCR Skew', 'RMSE', 'RMSE Median', 'RMSE Std Dev', 'RMSE Kurtosis', 'RMSE Skew']
df = pd.DataFrame(features, columns=columns)

1. columns: Variabel ini menyimpan list yang berisi nama-nama kolom yang akan digunakan untuk DataFrame. Kolom pertama disebut 'Label', dan sisanya adalah nama-nama statistik atau fitur yang dihitung sebelumnya.

2. df = pd.DataFrame(features, columns=columns): Membuat DataFrame menggunakan Pandas. Parameter features adalah list yang berisi hasil fitur atau statistik yang dikumpulkan sebelumnya, dan columns adalah nama kolom yang akan dijadikan DataFrame.






In [ ]:
# Menampilkan file CSV
df

,Label,ZCR Mean,ZCR Median,ZCR Std Dev,ZCR Kurtosis,ZCR Skew,RMSE,RMSE Median,RMSE Std Dev,RMSE Kurtosis,RMSE Skew
0,YAF_sad,0.128581,0.033203,0.225249,3.231511,2.213080,0.001907,0.000391,0.004361,28.695116,4.718993
1,YAF_sad,0.118958,0.028320,0.195816,4.225416,2.337965,0.000546,0.000059,0.001320,37.525358,5.243677
2,YAF_sad,0.124418,0.030762,0.203419,3.494584,2.197851,0.000502,0.000045,0.001103,20.908828,4.065221
3,YAF_sad,0.105494,0.035156,0.194269,5.715395,2.682275,0.001279,0.000270,0.002408,15.520610,3.484518
4,YAF_sad,0.138011,0.057129,0.193587,3.628069,2.167333,0.001019,0.000084,0.002397,23.264337,4.194062
...,...,...,...,...,...,...,...,...,...,...,...
2801,OAF_angry,0.083810,0.052246,0.099268,4.521899,2.239367,0.001529,0.000058,0.003707,21.352151,4.054795
2802,OAF_angry,0.115001,0.060059,0.118703,3.370580,2.012270,0.001743,0.000146,0.004667,41.713213,5.561090
2803,OAF_angry,0.083991,0.051270,0.091742,7.208814,2.744011,0.003030,0.000291,0.006712,21.150835,3.983283
2804,OAF_angry,0.085005,0.059082,0.090037,5.366187,2.422199,0.001496,0.000135,0.004290,31.532767,5.091522


In [ ]:
#menyimpan file hasil dari ekstraksi fitur
df.to_csv('dataemosi.csv',index=False)

# **NORMALISASI STANDARD SCALER**

StandardScaler adalah suatu metode dalam pemrosesan data yang digunakan untuk mentransformasi fitur-fitur data sehingga memiliki mean 0 dan deviasi standar 1. Ini adalah salah satu langkah umum dalam pra-pemrosesan data sebelum membangun model. Proses ini membantu menghilangkan perbedaan skala antar fitur, sehingga setiap fitur memiliki pengaruh yang sebanding pada model.


Rumus Standard Scaler:

z = (x - μ) / σ

di mana:
- z adalah nilai yang telah di-standarisasi,
- x adalah nilai asli,
- μ adalah rata-rata dari sampel,
- σ adalah deviasi standar dari sampel.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from pickle import dump

1. from sklearn.model_selection import train_test_split: Mengimpor fungsi train_test_split dari modul model_selection di scikit-learn. Fungsi ini digunakan untuk membagi dataset menjadi subset pelatihan dan pengujian.

2. from sklearn.preprocessing import StandardScaler: Mengimpor kelas StandardScaler dari modul preprocessing di scikit-learn. StandardScaler digunakan untuk melakukan penskalaan fitur sehingga fitur-fitur tersebut memiliki mean 0 dan varians 1.

3. from sklearn.neighbors import KNeighborsClassifier: Mengimpor kelas KNeighborsClassifier dari modul neighbors di scikit-learn. Kelas ini digunakan untuk membuat model klasifikasi dengan algoritma K-Nearest Neighbors.

4. from pickle import dump: Mengimpor fungsi dump dari modul pickle. dump digunakan untuk menyimpan objek Python ke dalam file, dan mungkin akan digunakan untuk menyimpan model klasifikasi setelah dilatih.

In [ ]:
# Baca data dari file CSV
dataknn = pd.read_csv('dataemosi.csv')

# Pisahkan fitur (X) dan label (y)
X = dataknn.drop(['Label'], axis=1)  # Menghapus kolom 'Label' sebagai fitur dan menyimpannya di variabel X
y = dataknn['Label']  # Mengambil kolom 'Label' sebagai label dan menyimpannya di variabel y

# Bagi data menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2)

# Tentukan scaler
scaler = StandardScaler()

# Terapkan scaler pada dataset pelatihan
scaler.fit(X_train)

# Simpan scaler
dump(scaler, open('scaler.pkl', 'wb'))

# Normalisasi dataset pelatihan
X_train_scaled = scaler.transform(X_train)

# Tampilkan datasetknn
dataknn


,Label,ZCR Mean,ZCR Median,ZCR Std Dev,ZCR Kurtosis,ZCR Skew,RMSE,RMSE Median,RMSE Std Dev,RMSE Kurtosis,RMSE Skew
0,YAF_sad,0.128581,0.033203,0.225249,3.231511,2.213080,0.001907,0.000391,0.004361,28.695116,4.718993
1,YAF_sad,0.118958,0.028320,0.195816,4.225416,2.337965,0.000546,0.000059,0.001320,37.525358,5.243677
2,YAF_sad,0.124418,0.030762,0.203419,3.494584,2.197851,0.000502,0.000045,0.001103,20.908828,4.065221
3,YAF_sad,0.105494,0.035156,0.194269,5.715395,2.682275,0.001279,0.000270,0.002408,15.520610,3.484518
4,YAF_sad,0.138011,0.057129,0.193587,3.628069,2.167333,0.001019,0.000084,0.002397,23.264337,4.194062
...,...,...,...,...,...,...,...,...,...,...,...
2801,OAF_angry,0.083810,0.052246,0.099268,4.521899,2.239367,0.001529,0.000058,0.003707,21.352151,4.054795
2802,OAF_angry,0.115001,0.060059,0.118703,3.370580,2.012270,0.001743,0.000146,0.004667,41.713213,5.561090
2803,OAF_angry,0.083991,0.051270,0.091742,7.208814,2.744011,0.003030,0.000291,0.006712,21.150835,3.983283
2804,OAF_angry,0.085005,0.059082,0.090037,5.366187,2.422199,0.001496,0.000135,0.004290,31.532767,5.091522


1. dataknn = pd.read_csv('dataemosi.csv'): Membaca data dari file CSV 'dataemosi.csv' dan menyimpannya dalam variabel dataknn.
2. X = dataknn.drop(['Label'], axis=1): Memisahkan fitur (kolom-kolom kecuali 'Label') dari dataset dan menyimpannya dalam variabel X.
3. y = dataknn['Label']: Menyimpan label (kolom 'Label') dalam variabel y.
4. X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2): Membagi data menjadi data latih dan data uji menggunakan train_test_split dengan random_state=1 dan ukuran uji sebesar 20% dari total data.
5. scaler = StandardScaler(): Membuat objek scaler untuk normalisasi fitur.
6. scaler.fit(X_train): Melatih scaler pada data latih.
7. dump(scaler, open('scaler.pkl', 'wb')): Menyimpan scaler ke dalam file 'scaler.pkl' untuk digunakan nanti.
8. X_train_scaled = scaler.transform(X_train): Normalisasi data latih menggunakan scaler yang telah dilatih.
9. dataknn: Menampilkan dataset dataknn setelah pemrosesan.

In [ ]:
import pickle
# Membuka file 'scaler.pkl' untuk membaca (mode 'rb' untuk mode baca biner)
with open('scaler.pkl', 'rb') as standarisasi:
    loadscal = pickle.load(standarisasi)


1. import pickle: Mengimpor modul pickle untuk membaca dan menulis objek Python dalam bentuk biner.
2. with open('scaler.pkl', 'rb') as standarisasi: Membuka file 'scaler.pkl' dalam mode membaca biner ('rb') dan menggunakan statement 'with' untuk memastikan bahwa file akan ditutup secara otomatis setelah blok kode di dalamnya selesai dieksekusi.
3. 'loadscal = pickle.load(standarisasi)': Menggunakan 'pickle.load()' untuk membaca objek scaler yang telah disimpan dalam file 'scaler.pkl' dan menyimpannya dalam variabel 'loadscal'.

In [ ]:
X_test_scaled=loadscal.transform(X_test) #normalisasi X testing dari hasil normalisasi X train yang disimpan dalam model

# **PEMODELAN**

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix

from sklearn.metrics import accuracy_score, confusion_matrix: Mengimpor fungsi accuracy_score dan confusion_matrix dari modul sklearn.metrics.
1. **accuracy_score**: Fungsi ini digunakan untuk mengukur tingkat akurasi dari suatu model klasifikasi. Akurasi adalah rasio jumlah prediksi yang benar terhadap total jumlah prediksi. Fungsi ini menerima dua parameter: nilai sebenarnya (y_true) dan nilai prediksi (y_pred), dan mengembalikan nilai akurasi.

2. **confusion_matrix**: Fungsi ini menghitung matriks kebingungan (confusion matrix) yang digunakan untuk mengevaluasi kinerja suatu model klasifikasi. Matriks kebingungan memberikan gambaran tentang seberapa baik model dapat mengklasifikasikan instans-instans pada kategori tertentu. Matriks ini memiliki empat sel, yaitu True Positive (TP), True Negative (TN), False Positive (FP), dan False Negative (FN). Fungsi ini juga menerima dua parameter: nilai sebenarnya (y_true) dan nilai prediksi (y_pred), dan mengembalikan matriks kebingungan dalam bentuk array.

In [ ]:
K = 30
acc = np.zeros((K-1))
# Loop untuk mencari nilai akurasi dengan jumlah tetangga (k) dari 1 hingga K dengan langkah 2
for n in range(1, K, 2):
    # Membuat model k-NN dengan nilai k tertentu dan metrik euclidean
    knn = KNeighborsClassifier(n_neighbors=n, metric="euclidean").fit(X_train_scaled, y_train)

    # Melakukan prediksi pada data uji
    y_pred = knn.predict(X_test_scaled)

    # Menghitung dan menyimpan akurasi untuk nilai k saat ini
    acc[n-1] = accuracy_score(y_test, y_pred)

# Menampilkan nilai akurasi terbaik dan nilai k yang sesuai
print('Akurasi terbaik adalah', acc.max(), 'dengan nilai k =', acc.argmax() + 1)


Akurasi terbaik adalah 0.7206405693950177 dengan nilai k = 13


1. K = 30: Mengatur nilai K (jumlah tetangga) maksimum yang akan dievaluasi.
2. acc = np.zeros((K-1)): Membuat array dengan panjang K-1 untuk menyimpan nilai akurasi pada setiap iterasi.
3. for n in range(1, K, 2):: Melakukan loop untuk nilai K dari 1 hingga K dengan langkah 2 (menggunakan bilangan ganjil).
4. knn = KNeighborsClassifier(n_neighbors=n, metric="euclidean").fit(X_train_scaled, y_train): Membuat model k-NN dengan jumlah tetangga (k) tertentu dan metrik Euclidean, lalu melatih model tersebut dengan data latih yang telah dinormalisasi.
5. y_pred = knn.predict(X_test_scaled): Melakukan prediksi pada data uji menggunakan model k-NN yang telah dilatih.
6. acc[n-1] = accuracy_score(y_test, y_pred): Menghitung dan menyimpan akurasi pada array acc untuk nilai k saat ini.
7. print('Akurasi terbaik adalah', acc.max(), 'dengan nilai k =', acc.argmax() + 1): Menampilkan nilai akurasi tertinggi dan nilai k yang sesuai.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=1000, random_state=42)
clf.fit(X_train_scaled, y_train)

RandomForestClassifier(n_estimators=1000, random_state=42)

1. from sklearn.ensemble import RandomForestClassifier: Mengimpor kelas RandomForestClassifier dari modul sklearn.ensemble.
2. clf = RandomForestClassifier(n_estimators=1000, random_state=42): Membuat objek RandomForestClassifier dengan 1000 pohon keputusan (n_estimators) dan seed random (random_state) yang ditetapkan ke 42.
3. clf.fit(X_train_scaled, y_train): Melatih model menggunakan data latih yang telah dinormalisasi (X_train_scaled) dan label data latih (y_train).

In [ ]:
# Melakukan prediksi pada data uji yang telah dinormalisasi menggunakan model RandomForestClassifier
y_pred = clf.predict(X_test_scaled)

# Menampilkan hasil prediksi
y_pred

array(['YAF_fear', 'YAF_pleasant_surprised', 'YAF_fear', 'YAF_angry',
       'YAF_angry', 'OAF_disgust', 'OAF_disgust', 'YAF_neutral',
       'OAF_disgust', 'OAF_neutral', 'YAF_fear', 'YAF_happy',
       'YAF_pleasant_surprised', 'OAF_happy', 'YAF_pleasant_surprised',
       'YAF_pleasant_surprised', 'OAF_fear', 'YAF_sad', 'YAF_angry',
       'YAF_disgust', 'YAF_disgust', 'OAF_neutral', 'YAF_neutral',
       'OAF_pleasant_surprise', 'YAF_sad', 'OAF_pleasant_surprise',
       'OAF_disgust', 'OAF_angry', 'YAF_pleasant_surprised', 'YAF_angry',
       'YAF_happy', 'OAF_pleasant_surprise', 'OAF_sad', 'OAF_fear',
       'YAF_disgust', 'OAF_happy', 'OAF_pleasant_surprise', 'OAF_neutral',
       'YAF_pleasant_surprised', 'YAF_pleasant_surprised', 'OAF_sad',
       'OAF_pleasant_surprise', 'YAF_neutral', 'OAF_fear', 'OAF_fear',
       'YAF_disgust', 'YAF_sad', 'OAF_pleasant_surprise', 'YAF_disgust',
       'OAF_sad', 'OAF_sad', 'OAF_disgust', 'YAF_disgust', 'YAF_fear',
       'YAF_pleasant_surp

1. y_pred = clf.predict(X_test_scaled): Menggunakan model RandomForestClassifier yang telah dilatih untuk melakukan prediksi pada data uji yang telah dinormalisasi (X_test_scaled). Hasil prediksi disimpan dalam variabel y_pred.
2. y_pred: Menampilkan hasil prediksi, yang merupakan prediksi kelas untuk setiap sampel dalam data uji.

In [ ]:
# Menghitung akurasi antara nilai sebenarnya dan nilai prediksi
accuracy = accuracy_score(y_test, y_pred)
# Menampilkan nilai akurasi
print("Akurasi:", accuracy)

Akurasi: 0.802491103202847


1. accuracy = accuracy_score(y_test, y_pred): Menggunakan fungsi accuracy_score dari modul sklearn.metrics untuk menghitung akurasi antara nilai sebenarnya (y_test) dan nilai prediksi (y_pred).
2. print("Akurasi:", accuracy): Menampilkan nilai akurasi pada output. Akurasi merupakan rasio jumlah prediksi yang benar terhadap total jumlah prediksi pada data uji.

In [ ]:
dump(knn, open('m_knn.pkl', 'wb'))

dump(knn, open('m_knn.pkl', 'wb')): Menggunakan fungsi dump dari modul pickle untuk menyimpan model k-NN (knn) ke dalam file 'm_knn.pkl' dalam mode penulisan biner ('wb'). Ini akan menghasilkan file yang berisi model k-NN yang dapat digunakan kembali di lain waktu.

# **PCA**


Principal Component Analysis (PCA) adalah suatu metode dalam statistika multivariat yang digunakan untuk mengubah dataset yang kompleks menjadi bentuk yang lebih sederhana dan lebih mudah diinterpretasikan. Tujuan utama dari PCA adalah mereduksi dimensi data, sehingga kita dapat mempertahankan sebanyak mungkin informasi yang dapat dijelaskan oleh sejumlah komponen utama atau principal components.

In [ ]:
from sklearn.decomposition import PCA as sklearnPCA

# Membuat objek PCA dengan 9 komponen utama
sklearn_pca = sklearnPCA(n_components=9)

# Melakukan transformasi PCA pada data latih yang telah dinormalisasi
X_train_pca = sklearn_pca.fit_transform(X_train_scaled)

# Menampilkan tipe data dari X_train_pca
type(X_train_pca)

numpy.ndarray

1. from sklearn.decomposition import PCA as sklearnPCA: Mengimpor kelas PCA dari modul sklearn.decomposition dan memberikan alias sklearnPCA.
2. sklearn_pca = sklearnPCA(n_components=9): Membuat objek PCA dengan mengatur jumlah komponen utama menjadi 9.
3. X_train_pca = sklearn_pca.fit_transform(X_train_scaled): Melakukan transformasi PCA pada data latih yang telah dinormalisasi (X_train_scaled) dan menyimpan hasilnya dalam X_train_pca.
4. type(X_train_pca): Menampilkan tipe data dari X_train_pca. Hasilnya seharusnya berupa numpy array atau matriks hasil reduksi dimensi menggunakan PCA.

In [ ]:
dump(sklearn_pca, open('PCArf9.pkl', 'wb'))

dump(sklearn_pca, open('PCArf9.pkl', 'wb')): Menggunakan fungsi dump dari modul pickle untuk menyimpan model PCA (sklearn_pca) ke dalam file 'PCArf9.pkl' dalam mode penulisan biner ('wb'). Ini akan menghasilkan file yang berisi model PCA dengan konfigurasi 9 komponen utama yang dapat digunakan kembali di lain waktu.

In [ ]:
import pickle
with open('PCArf9.pkl', 'rb') as pca:
   loadpca= pickle.load(pca)

1. import pickle: Mengimpor modul pickle untuk membaca dan menulis objek Python dalam bentuk biner.
2. with open('PCArf9.pkl', 'rb') as pca:: Membuka file 'PCArf9.pkl' dalam mode membaca biner ('rb') dan menggunakan statement with untuk memastikan bahwa file akan ditutup secara otomatis setelah blok kode di dalamnya selesai dieksekusi.
3. loadpca = pickle.load(pca): Menggunakan pickle.load() untuk membaca objek PCA yang telah disimpan dalam file 'PCArf9.pkl' dan menyimpannya dalam variabel loadpca.

In [ ]:
# Melakukan transformasi PCA pada data uji yang telah dinormalisasi menggunakan model PCA yang telah dimuat
X_test_pca = loadpca.transform(X_test_scaled)

# Menampilkan bentuk (shape) dari X_test_pca
X_test_pca.shape


(562, 9)

1. X_test_pca = loadpca.transform(X_test_scaled): Menggunakan model PCA yang telah dimuat (loadpca) untuk melakukan transformasi PCA pada data uji yang telah dinormalisasi (X_test_scaled) dan menyimpan hasilnya dalam X_test_pca.
2. X_test_pca.shape: Menampilkan bentuk (shape) dari X_test_pca, yang akan memberikan informasi tentang jumlah baris dan kolom dalam matriks hasil transformasi PCA.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Membuat objek RandomForestClassifier dengan 1000 pohon keputusan dan seed random 42
clfr = RandomForestClassifier(n_estimators=1000, random_state=42)
# Melatih model menggunakan data latih yang telah mengalami reduksi dimensi PCA
clfr.fit(X_train_pca, y_train)

RandomForestClassifier(n_estimators=1000, random_state=42)

1. from sklearn.ensemble import RandomForestClassifier: Mengimpor kelas RandomForestClassifier dari modul sklearn.ensemble.
2. clfr = RandomForestClassifier(n_estimators=1000, random_state=42): Membuat objek RandomForestClassifier dengan 1000 pohon keputusan (n_estimators) dan seed random (random_state) yang ditetapkan ke 42.
3. clfr.fit(X_train_pca, y_train): Melatih model menggunakan data latih yang telah mengalami reduksi dimensi PCA (X_train_pca) dan label data latih (y_train). Model RandomForestClassifier akan belajar dari data ini untuk membuat prediksi pada tahap selanjutnya.

In [ ]:
# Melakukan prediksi pada data uji yang telah mengalami reduksi dimensi PCA menggunakan model RandomForestClassifier
y_prediksi = clfr.predict(X_test_pca)

# Menampilkan hasil prediksi
y_prediksi


array(['YAF_fear', 'YAF_pleasant_surprised', 'YAF_fear', 'YAF_angry',
       'YAF_angry', 'OAF_disgust', 'OAF_pleasant_surprise', 'YAF_neutral',
       'OAF_disgust', 'OAF_neutral', 'YAF_fear', 'YAF_happy',
       'YAF_pleasant_surprised', 'OAF_pleasant_surprise',
       'YAF_pleasant_surprised', 'YAF_pleasant_surprised', 'OAF_fear',
       'YAF_sad', 'YAF_angry', 'YAF_disgust', 'YAF_disgust',
       'OAF_neutral', 'YAF_neutral', 'OAF_pleasant_surprise', 'YAF_sad',
       'OAF_disgust', 'OAF_disgust', 'OAF_angry',
       'YAF_pleasant_surprised', 'YAF_angry', 'YAF_happy',
       'OAF_pleasant_surprise', 'OAF_sad', 'OAF_fear', 'YAF_disgust',
       'OAF_happy', 'OAF_fear', 'OAF_neutral', 'YAF_pleasant_surprised',
       'YAF_neutral', 'OAF_sad', 'OAF_pleasant_surprise', 'YAF_neutral',
       'OAF_fear', 'OAF_fear', 'YAF_disgust', 'YAF_sad', 'OAF_fear',
       'YAF_disgust', 'OAF_happy', 'OAF_sad', 'OAF_disgust',
       'YAF_disgust', 'YAF_fear', 'YAF_pleasant_surprised', 'OAF_sad',
    

1. y_prediksi = clfr.predict(X_test_pca): Menggunakan model RandomForestClassifier yang telah dilatih untuk melakukan prediksi pada data uji yang telah mengalami reduksi dimensi PCA (X_test_pca). Hasil prediksi disimpan dalam variabel y_prediksi.
2. y_prediksi: Menampilkan hasil prediksi, yang merupakan prediksi kelas untuk setiap sampel dalam data uji yang telah mengalami reduksi dimensi PCA.

In [ ]:
acc_pca= accuracy_score(y_test,y_prediksi)
print("Akurasi:",acc_pca)

Akurasi: 0.7793594306049823


1. acc_pca = accuracy_score(y_test, y_prediksi): Menggunakan fungsi accuracy_score dari modul sklearn.metrics untuk menghitung akurasi antara nilai sebenarnya (y_test) dan nilai prediksi setelah reduksi dimensi PCA (y_prediksi).
2. print("Akurasi:", acc_pca): Menampilkan nilai akurasi pada output. Akurasi adalah rasio jumlah prediksi yang benar terhadap total jumlah prediksi pada data uji setelah mengalami reduksi dimensi PCA.

In [ ]:
dump(clfr, open('knnpca9.pkl', 'wb'))

dump(clfr, open('knnpca9.pkl', 'wb')): Menggunakan fungsi dump dari modul pickle untuk menyimpan model klasifikasi setelah reduksi dimensi PCA (clfr) ke dalam file 'knnpca9.pkl'